In [1]:
import pandas as pd
import json
import numpy as np
import ast
import locale


In [24]:
# se cargan los datos del diccionario
dicc_df= pd.read_excel('Diccionario de Datos STEAM.xlsx' )

In [25]:
#mostramos las primeras lineas del DataFrame
dicc_df.head()

,steam_games.gz.json,Unnamed: 1,Unnamed: 2
0,Columna,Descripción,Ejemplo
1,publisher,Empresa publicadora del contenido,"[Ubisoft,Dovetail Games - Trains,Degica]"
2,genres,Genero del contenido,"[Action, Adventure, Racing, Simulation, Strategy]"
3,app_name,Nombre del contenido,"[Warzone, Soundtrack, Puzzle Blocks]"
4,title,Titulo del contenido,"[The Dream Machine: Chapter 4 , Fate/EXTELLA -..."


In [26]:
#se cargan los datos del formato json

with open ("australian_user_reviews.json", 'r', encoding='utf-8') as file:
    data_reviews= file.readlines()

# se convierten las lineas
reviews = [eval(line.strip()) for line in data_reviews]


In [27]:
#creamos un DF a partir de los registros

df_reviews= pd.DataFrame(reviews)

#mostramos las primeras filas
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [28]:
#reviews_df2= df_reviews['reviews'] # pasamos a un dataframe la columna reviews

#reviews_df2.to_csv('reviews_columna.csv', index=False) #convertí la columna reviews a csv para poder ver solamente la columna de reviews

reviews_df2= df_reviews.explode('reviews') #quitamos los corchetes 

In [29]:
# lista de para anexar más adelante con el nombre en la columna review

funny=       []
posted=      []
last_edited= []
item_id=     []
helpful=     []
recommend=   []
review=      []

# se recorre cada fila del DF 
for index,row in reviews_df2.iterrows():

    #se obtiene cada valor en la columna de 'reviews' para la fila actual
    elem= row['reviews']

    #verificamos si el valor que tenemos es un diccionario
    if isinstance(elem, dict):

        #extraemos los valores y se agregan a la lista correspondiente

        funny.append(elem.get('funny','')) #se agrega el nombre o un espacio en blanco si no se encuentra
        posted.append(elem.get('posted', ''))
        last_edited.append(elem.get('last_edited', ''))
        item_id.append(elem.get('item_id', ''))
        helpful.append(elem.get('helpful', ''))
        recommend.append(elem.get('recommend'))
        review.append(elem.get('review',''))

    else:
        # de no haber diccionario se agregan espacios en blanco
        funny.append('')
        posted.append('')
        last_edited.append('')
        item_id.append('')
        helpful.append('')
        recommend.append('')
        review.append('')

# aquí se agregaran las listas como una columna nueva en el DF
reviews_df2['Funny']= funny
reviews_df2['Posted']= posted
reviews_df2['Last edited']=last_edited
reviews_df2['Item id'] = item_id
reviews_df2['Helpful']= helpful
reviews_df2['Recommend']= recommend
reviews_df2['Review']= review


In [31]:
reviews_df2.drop('reviews', axis=1, inplace=True)

In [38]:
reviews_df2.head() #comprobamos que se guardo correctamente

,user_id,user_url,Funny,Posted,Item id,Helpful,Recommend,Review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,1250,NaN,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,22200,NaN,True,It's unique and worth a playthrough.
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,43110,NaN,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,http://steamcommunity.com/id/js41637,,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [33]:
reviews_df2.drop('Last edited', axis=1, inplace=True) #eliminamos esta columna 

In [35]:
# se cambia el valor de no ratings yet a nulos en la columna de helpful
reviews_df2['Helpful']= np.where(reviews_df2['Helpful']== 'No ratings yet', np.nan, reviews_df2['Helpful']) 

In [37]:
#definimos una variable para encontrar el patron de las fechas en la tabla reviews

pattern= r'Posted (\w+ \d+, \d{4})'

#se filtra y extran las fechas 

reviews_df2['Posted']= reviews_df2['Posted'].str.extract(pattern)[0]

#convertimos las fechas en formato de date
reviews_df2['Posted']= pd.to_datetime(reviews_df2['Posted'], format='%B %d, %Y')


In [11]:
reviews_df2.drop('Review', axis=1, inplace=True) #eliminamos columna inecesaria

In [16]:
#pasamos a df el archivo json
df_games= pd.read_json("output_steam_games.json", lines=True)

In [19]:
#vemos las ultimas filas para comprobar el dataframe
df_games.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [18]:
df_games_nonulos= df_games.dropna(how='all')

In [62]:
df_games_nonulos.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


In [61]:
#normalizamos precios

def convertir_cero(v):
    if isinstance(v,str):
        return 0
    return v

#aplicamos la funcion a la columna price del DF

df_games_nonulos.loc[:,'price']=df_games_nonulos['price'].apply(convertir_cero)

In [63]:
#convertimos a flotantes
df_games_nonulos['price'].astype(float)

88310     4.99
88311     0.00
88312     0.00
88313     0.99
88314     2.99
          ... 
120441    4.99
120442    1.99
120443    4.99
120444    4.99
price      NaN
Name: price, Length: 32136, dtype: float64

In [50]:
df_games_nonulos.to_csv('Steam_games.csv', index=True)

In [21]:
#se cargan los datos del formato json de los items

with open ("australian_users_items.json", 'r', encoding='utf8') as file:
    data_items= file.readlines()

# se convierten las lineas de items
items = [eval(line.strip()) for line in data_items]

In [22]:
#se transforma a dataFrame 
df_items= pd.DataFrame(items)


In [23]:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [40]:
df_items.explode('items') #se quitan los corchetes 

#creamos listas para más adelante poder agregarlas como columnas

item_id =          []
item_name =        []
playtime_forever = []
playtime_2weeks =  []

#recorremos cada fila del DF 
for index, row in df_items.iterrows():
    elem= row['items']

    #verificamos si el valor es un diccionario
    if isinstance(elem, dict):

        item_id.append(elem.get('item_id', None)) #agregamos el item o none si no está presente
        item_name.append(elem.get('item_name', None))
        playtime_forever.append(elem.get('playtime_forever', None))
        playtime_2weeks.append(elem.get('playtime_2weeks', None))

    else:
        #de no encontrarse en el diccionario se agregará un None a todas las listas
        item_id.append(None)
        item_name.append(None)
        playtime_forever.append(None)
        playtime_2weeks.append(None)

#agregamos las listas al DF

df_items['item_id']= item_id
df_items['item_name']= item_name
df_items['playtime_forever']= playtime_forever
df_items['playtime_2weeks']= playtime_2weeks


In [46]:
df_items.head(100)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,None,None,None
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,None,None,None,None
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,None,None,None,None
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,None,None,None,None
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,None,None,None,None
...,...,...,...,...,...,...,...,...
95,firefreddy,25,76561198059463642,http://steamcommunity.com/id/firefreddy,None,None,None,None
96,UTNerd24,188,76561198053985682,http://steamcommunity.com/id/UTNerd24,None,None,None,None
97,sandwiches1,172,76561198052850911,http://steamcommunity.com/id/sandwiches1,None,None,None,None
98,76561198093083895,15,76561198093083895,http://steamcommunity.com/profiles/76561198093...,None,None,None,None


In [42]:
df_items.drop('items', axis=1, inplace=True)

In [48]:
df_items.to_csv('items.csv', index=True)

In [49]:
reviews_df2.to_csv('reviews.csv', index=True)